In [1]:
# ================================================================
# Assignment 05 – Population Analysis by Sex and Age in Ireland
# ================================================================
# Import required libraries
import pandas as pd
import numpy as np

# Set display options to see all rows/columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)


In [19]:
# -------------------------------
# Step 1: Load the new dataset
# -------------------------------

import pandas as pd

# Path to the new CSV
data_url = 'ByAgeAndGender.csv'

# Load CSV into DataFrame
df = pd.read_csv(data_url)

# Inspect first few rows and column names
df.head(10)
print(df.columns)


Index(['Statistic Label', 'CensusYear', 'Sex', 'Single Year of Age',
       'County and City', 'UNIT', 'VALUE'],
      dtype='object')


In [20]:
# -------------------------------
# Step 2: Clean dataset for analysis
# -------------------------------

# Keep only Male and Female rows
df_clean = df[df['Sex'].isin(['Male', 'Female'])].copy()

# Convert 'Single Year of Age' to numeric
df_clean['Single Year of Age'] = pd.to_numeric(df_clean['Single Year of Age'], errors='coerce')

# Convert VALUE to numeric (remove commas if present)
df_clean['VALUE'] = pd.to_numeric(df_clean['VALUE'].astype(str).str.replace(',', ''), errors='coerce')

# Drop any rows with missing age or population
df_clean = df_clean.dropna(subset=['Single Year of Age', 'VALUE'])

# Reset index
df_clean = df_clean.reset_index(drop=True)

# Inspect cleaned dataset
df_clean.head(10)


,Statistic Label,CensusYear,Sex,Single Year of Age,County and City,UNIT,VALUE


In [40]:
# -------------------------------
# Step 3: Weighted mean age by sex
# -------------------------------

# Aggregate population by Sex and Age across all regions
df_agg = df_clean.groupby(['Sex', 'Age'], as_index=False)['VALUE'].sum()

# Calculate weighted mean age
weighted_mean_age = df_agg.groupby('Sex').apply(
    lambda x: (x['Age'] * x['VALUE']).sum() / x['VALUE'].sum()
)

print("Weighted mean age by sex:")
print(weighted_mean_age)


Weighted mean age by sex:
Sex
Female    37.611700
Male      36.379619
dtype: float64


/var/folders/j5/v1w8vnjn5j5bfl94cbr3dmv40000gn/T/ipykernel_67070/2255298561.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_mean_age = df_agg.groupby('Sex').apply(


In [39]:
# -------------------------------
# Step 4: Difference between sexes by age
# -------------------------------

# Pivot the DataFrame so we have columns for Male and Female
df_pivot = df.pivot_table(index='Single Year of Age', columns='Sex', values='VALUE', aggfunc='sum').reset_index()

# Calculate the difference: Male - Female
df_pivot['Difference'] = df_pivot['Male'] - df_pivot['Female']

# Inspect the first few rows
df_pivot.head()


Sex,Single Year of Age,Both sexes,Female,Male,Difference
0,1 year,386188,188330,197858,9528
1,10 years,400646,195214,205432,10218
2,100 years and over,3166,2574,592,-1982
3,11 years,400014,195162,204852,9690
4,12 years,403626,197060,206566,9506


In [25]:
# Check unique values in Sex column
print(df_agg['Sex'].unique())

[]


In [26]:
print(df_clean[['Sex', 'Single Year of Age']].head(20))
print(df_clean['Sex'].unique())
print(df_clean['Single Year of Age'].unique())


Empty DataFrame
Columns: [Sex, Single Year of Age]
Index: []
[]
[]


In [27]:
# Inspect first 50 rows of Sex and Single Year of Age
print(df[['Sex', 'Single Year of Age']].head(50))

# See unique values in Sex column
print(df['Sex'].unique())

# See unique values in Single Year of Age column
print(df['Single Year of Age'].unique())


           Sex Single Year of Age
0   Both sexes           All ages
1   Both sexes           All ages
2   Both sexes           All ages
3   Both sexes           All ages
4   Both sexes           All ages
5   Both sexes           All ages
6   Both sexes           All ages
7   Both sexes           All ages
8   Both sexes           All ages
9   Both sexes           All ages
10  Both sexes           All ages
11  Both sexes           All ages
12  Both sexes           All ages
13  Both sexes           All ages
14  Both sexes           All ages
15  Both sexes           All ages
16  Both sexes           All ages
17  Both sexes           All ages
18  Both sexes           All ages
19  Both sexes           All ages
20  Both sexes           All ages
21  Both sexes           All ages
22  Both sexes           All ages
23  Both sexes           All ages
24  Both sexes           All ages
25  Both sexes           All ages
26  Both sexes           All ages
27  Both sexes           All ages
28  Both sexes

In [28]:
def convert_age(age_str):
    if age_str == 'Under 1 year':
        return 0
    elif age_str == '100 years and over':
        return 100
    else:
        # Remove ' year' or ' years' and convert to int
        return int(age_str.split(' ')[0])

# Apply conversion
df_clean['Age'] = df_clean['Single Year of Age'].apply(convert_age)



In [29]:
df_clean = df[df['Sex'].isin(['Male', 'Female'])].copy()


In [37]:
# -------------------------------
# Step 2b: Convert age strings to numeric
# -------------------------------

# Work on df_clean (which already has only Male/Female rows)
def convert_age(age_str):
    if age_str == 'Under 1 year':
        return 0
    elif age_str == '100 years and over':
        return 100
    else:
        # Remove ' year' or ' years' and convert to int
        return int(age_str.split(' ')[0])

# Apply conversion and store in a new column
df_clean['Age'] = df_clean['Single Year of Age'].apply(convert_age)

# Inspect to make sure the new column exists
print(df_clean[['Sex', 'Single Year of Age', 'Age', 'VALUE']].head(10))



       Sex Single Year of Age  Age  VALUE
3193  Male       Under 1 year    0  36850
3194  Male       Under 1 year    0   4185
3195  Male       Under 1 year    0    837
3196  Male       Under 1 year    0    635
3197  Male       Under 1 year    0    469
3198  Male       Under 1 year    0   3530
3199  Male       Under 1 year    0   1302
3200  Male       Under 1 year    0   1409
3201  Male       Under 1 year    0   2751
3202  Male       Under 1 year    0    595


In [32]:
# -------------------------------
# Step 2b: Filter and convert age strings to numeric
# -------------------------------

# Keep only Male/Female and exclude 'All ages'
df_clean = df[(df['Sex'].isin(['Male', 'Female'])) & (df['Single Year of Age'] != 'All ages')].copy()

# Define conversion function
def convert_age(age_str):
    if age_str == 'Under 1 year':
        return 0
    elif age_str == '100 years and over':
        return 100
    else:
        # Remove ' year' or ' years' and convert to int
        return int(age_str.split(' ')[0])

# Apply conversion
df_clean['Age'] = df_clean['Single Year of Age'].apply(convert_age)

# Inspect to verify
print(df_clean[['Sex', 'Single Year of Age', 'Age', 'VALUE']].head(10))
print(df_clean['Age'].unique())
print(df_clean['Sex'].unique())


       Sex Single Year of Age  Age  VALUE
3193  Male       Under 1 year    0  36850
3194  Male       Under 1 year    0   4185
3195  Male       Under 1 year    0    837
3196  Male       Under 1 year    0    635
3197  Male       Under 1 year    0    469
3198  Male       Under 1 year    0   3530
3199  Male       Under 1 year    0   1302
3200  Male       Under 1 year    0   1409
3201  Male       Under 1 year    0   2751
3202  Male       Under 1 year    0    595
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100]
['Male' 'Female']


In [33]:
# -------------------------------
# Step 3: Weighted mean age by sex
# -------------------------------

# Aggregate population by Sex and Age across all regions
df_agg = df_clean.groupby(['Sex', 'Age'], as_index=False)['VALUE'].sum()

# Calculate weighted mean age
weighted_mean_age = df_agg.groupby('Sex').apply(
    lambda x: (x['Age'] * x['VALUE']).sum() / x['VALUE'].sum()
)

print("Weighted mean age by sex:")
print(weighted_mean_age)



Weighted mean age by sex:
Sex
Female    37.611700
Male      36.379619
dtype: float64


/var/folders/j5/v1w8vnjn5j5bfl94cbr3dmv40000gn/T/ipykernel_67070/1828908023.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_mean_age = df_agg.groupby('Sex').apply(


In [34]:
# -------------------------------
# Step 4: Difference between sexes by age
# -------------------------------

# Pivot df_agg so that 'Male' and 'Female' are columns
df_pivot = df_agg.pivot(index='Age', columns='Sex', values='VALUE').reset_index()

# Calculate the difference: Male - Female
df_pivot['Difference'] = df_pivot['Male'] - df_pivot['Female']

# Inspect first 10 rows
df_pivot.head(10)


Sex,Age,Female,Male,Difference
0,0,188254,196672,8418
1,1,188330,197858,9528
2,2,193348,202840,9492
3,3,195698,204336,8638
4,4,194458,204724,10266
5,5,197306,205428,8122
6,6,199096,209318,10222
7,7,200994,210222,9228
8,8,201510,210636,9126
9,9,197142,206350,9208


In [35]:
# -------------------------------
# Part 2: Population difference in a 5-year age group
# -------------------------------

# Define the target age
target_age = 35

# Define age range: within 5 years of target_age
age_min = target_age - 2
age_max = target_age + 2

# Filter df_pivot for ages within this range
age_group = df_pivot[(df_pivot['Age'] >= age_min) & (df_pivot['Age'] <= age_max)]

# Calculate total Male and Female populations in this group
total_male = age_group['Male'].sum()
total_female = age_group['Female'].sum()

# Calculate difference
difference = total_male - total_female

print(f"Age group: {age_min}-{age_max}")
print(f"Total Male: {total_male}, Total Female: {total_female}")
print(f"Population difference (Male - Female): {difference}")


Age group: 33-37
Total Male: 1098034, Total Female: 1153766
Population difference (Male - Female): -55732


In [36]:
# -------------------------------
# Part 3: Region with biggest difference in the 5-year age group
# -------------------------------

# Filter the cleaned dataset for the age range
df_region_age_group = df_clean[(df_clean['Age'] >= age_min) & (df_clean['Age'] <= age_max)]

# Aggregate by region (County and City) and Sex
df_region_agg = df_region_age_group.groupby(['County and City', 'Sex'], as_index=False)['VALUE'].sum()

# Pivot so we have columns for Male and Female
df_region_pivot = df_region_agg.pivot(index='County and City', columns='Sex', values='VALUE').fillna(0)

# Calculate difference: Male - Female
df_region_pivot['Difference'] = df_region_pivot['Male'] - df_region_pivot['Female']

# Find the region with the largest absolute difference
max_diff_region = df_region_pivot['Difference'].abs().idxmax()
max_diff_value = df_region_pivot.loc[max_diff_region, 'Difference']

print("Region with biggest population difference:")
print(f"{max_diff_region} with difference {max_diff_value}")



Region with biggest population difference:
State with difference -27866
